In [63]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [64]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import seaborn as sns

In [65]:
train = pd.read_csv("/content/drive/MyDrive/Project Data Mining/Predicting Coupon Redemption/train.csv")
demographics = pd.read_csv("/content/drive/MyDrive/Project Data Mining/Predicting Coupon Redemption/customer_demographics.csv")
transaction = pd.read_csv("/content/drive/MyDrive/Project Data Mining/Predicting Coupon Redemption/customer_transaction_data.csv", parse_dates=['date'])
coupon_item = pd.read_csv("/content/drive/MyDrive/Project Data Mining/Predicting Coupon Redemption/coupon_item_mapping.csv")
item = pd.read_csv("/content/drive/MyDrive/Project Data Mining/Predicting Coupon Redemption/item_data.csv")
campaign = pd.read_csv("/content/drive/MyDrive/Project Data Mining/Predicting Coupon Redemption/campaign_data.csv", parse_dates=['start_date','end_date'])

In [66]:
train.head()

,id,campaign_id,coupon_id,customer_id,redemption_status
0,1,13,27,1053,0
1,2,13,116,48,0
2,6,9,635,205,0
3,7,13,644,1050,0
4,9,8,1017,1489,0


In [67]:
train.nunique()

id                   78369
campaign_id             18
coupon_id              866
customer_id           1428
redemption_status        2
dtype: int64

In [68]:
campaign

,campaign_id,campaign_type,start_date,end_date
0,24,Y,2013-10-21,2013-12-20
1,25,Y,2013-10-21,2013-11-22
2,20,Y,2013-07-09,2013-11-16
3,23,Y,2013-08-10,2013-11-15
4,21,Y,2013-09-16,2013-10-18
5,22,X,2013-09-16,2013-10-18
6,18,X,2013-10-08,2013-04-10
7,19,Y,2013-08-26,2013-09-27
8,17,Y,2013-07-29,2013-08-30
9,16,Y,2013-07-15,2013-08-16


In [69]:
train.head()

,id,campaign_id,coupon_id,customer_id,redemption_status
0,1,13,27,1053,0
1,2,13,116,48,0
2,6,9,635,205,0
3,7,13,644,1050,0
4,9,8,1017,1489,0


In [70]:
d1 = train.groupby('campaign_id').agg(redemption_frequency=('redemption_status','sum'))
d1

,redemption_frequency
campaign_id,
1,1
2,5
3,2
4,7
5,7
6,1
7,4
8,199
9,23


In [71]:
d2 = train['campaign_id'].value_counts().sort_index().rename('total_amount').to_frame()
d2.index.name = d1.index.name
d2

,total_amount
campaign_id,
1,143
2,768
3,408
4,972
5,1827
6,65
7,1584
8,22555
9,3168


In [72]:
d2.iloc[:, 0].sum() == len(train)

True

In [73]:
coupon_types = train.coupon_id.nunique()
coupon_types

866

In [74]:
d3 = train.groupby('campaign_id')['coupon_id'].nunique()
d3

campaign_id
1      11
2      16
3      34
4      12
5      11
6       1
7       8
8     208
9      18
10     14
11     13
12     15
13    207
26    181
27     27
28     28
29     33
30    178
Name: coupon_id, dtype: int64

In [75]:
d3 = (d3 / coupon_types * 100).rename('coupon_covering_ratio%')
d3

campaign_id
1      1.270208
2      1.847575
3      3.926097
4      1.385681
5      1.270208
6      0.115473
7      0.923788
8     24.018476
9      2.078522
10     1.616628
11     1.501155
12     1.732102
13    23.903002
26    20.900693
27     3.117783
28     3.233256
29     3.810624
30    20.554273
Name: coupon_covering_ratio%, dtype: float64

In [76]:
d4 = train.groupby('campaign_id')['customer_id'].nunique()
d4

campaign_id
1       13
2       48
3       12
4       81
5      166
6       65
7      198
8     1076
9      176
10     123
11     214
12     170
13    1077
26     332
27      12
28      17
29     118
30     361
Name: customer_id, dtype: int64

In [77]:
n_customers = train.customer_id.nunique()
n_customers

1428

In [78]:
d4 = train.groupby('campaign_id')['customer_id'].nunique()
d4

campaign_id
1       13
2       48
3       12
4       81
5      166
6       65
7      198
8     1076
9      176
10     123
11     214
12     170
13    1077
26     332
27      12
28      17
29     118
30     361
Name: customer_id, dtype: int64

In [79]:
d4 = (d4 / n_customers * 100).rename('customer_covering_ratio%')
d4

campaign_id
1      0.910364
2      3.361345
3      0.840336
4      5.672269
5     11.624650
6      4.551821
7     13.865546
8     75.350140
9     12.324930
10     8.613445
11    14.985994
12    11.904762
13    75.420168
26    23.249300
27     0.840336
28     1.190476
29     8.263305
30    25.280112
Name: customer_covering_ratio%, dtype: float64

In [80]:
d = pd.concat([d1, d2, d3, d4], axis=1)
d['redemption_ratio%'] = d['redemption_frequency'] / d['total_amount'] * 100
d = d.drop(columns=['redemption_frequency'])

In [81]:
campaign_origin = campaign.copy()
campaign_origin

,campaign_id,campaign_type,start_date,end_date
0,24,Y,2013-10-21,2013-12-20
1,25,Y,2013-10-21,2013-11-22
2,20,Y,2013-07-09,2013-11-16
3,23,Y,2013-08-10,2013-11-15
4,21,Y,2013-09-16,2013-10-18
5,22,X,2013-09-16,2013-10-18
6,18,X,2013-10-08,2013-04-10
7,19,Y,2013-08-26,2013-09-27
8,17,Y,2013-07-29,2013-08-30
9,16,Y,2013-07-15,2013-08-16


In [82]:
campaign = campaign_origin.copy()

In [83]:
for index, row in campaign.iterrows():
  if row.end_date < row.start_date:
    campaign.iloc[index, 2] = row.end_date
    campaign.iloc[index, 3] = row.start_date

campaign

,campaign_id,campaign_type,start_date,end_date
0,24,Y,2013-10-21,2013-12-20
1,25,Y,2013-10-21,2013-11-22
2,20,Y,2013-07-09,2013-11-16
3,23,Y,2013-08-10,2013-11-15
4,21,Y,2013-09-16,2013-10-18
5,22,X,2013-09-16,2013-10-18
6,18,X,2013-04-10,2013-10-08
7,19,Y,2013-08-26,2013-09-27
8,17,Y,2013-07-29,2013-08-30
9,16,Y,2013-07-15,2013-08-16


In [84]:
campaign['duration_days'] = (campaign['end_date'] - campaign['start_date']).dt.days

campaign['start_year'] = campaign['start_date'].dt.year
campaign['start_quarter'] = campaign.start_date.dt.quarter
campaign['start_week'] = campaign.start_date.dt.isocalendar().week
campaign['start_weekday'] = campaign.start_date.dt.weekday

campaign['end_year'] = campaign['end_date'].dt.year
campaign['end_quarter'] =  campaign.end_date.dt.quarter
campaign['end_week'] =  campaign.end_date.dt.isocalendar().week
campaign['end_weekday'] =  campaign.end_date.dt.weekday

In [85]:
df = campaign.drop(columns=['start_date', 'end_date'])

In [86]:
df

,campaign_id,campaign_type,duration_days,start_year,start_quarter,start_week,start_weekday,end_year,end_quarter,end_week,end_weekday
0,24,Y,60,2013,4,43,0,2013,4,51,4
1,25,Y,32,2013,4,43,0,2013,4,47,4
2,20,Y,130,2013,3,28,1,2013,4,46,5
3,23,Y,97,2013,3,32,5,2013,4,46,4
4,21,Y,32,2013,3,38,0,2013,4,42,4
5,22,X,32,2013,3,38,0,2013,4,42,4
6,18,X,181,2013,2,15,2,2013,4,41,1
7,19,Y,32,2013,3,35,0,2013,3,39,4
8,17,Y,32,2013,3,31,0,2013,3,35,4
9,16,Y,32,2013,3,29,0,2013,3,33,4


In [59]:
# for col in ['end_year', 'start_year', 'campaign_type']:
#   dummy = pd.get_dummies(df[col], prefix=col, prefix_sep='_', drop_first=True, dummy_na=False)
#   df = pd.concat([dummy, df.drop(columns=col)], axis=1)

In [87]:
df = df.merge(d, on='campaign_id', how='right')

In [88]:
df['amount/day'] = df['total_amount'] / df['duration_days']

In [90]:
df["redemption_ratio"] = df['redemption_ratio%']
df = df.drop(columns = ['redemption_ratio%'])
df.rename(columns={"redemption_ratio":'redemption_ratio%'}, inplace=True)
df.to_markdown()

'|    |   campaign_id | campaign_type   |   duration_days |   start_year |   start_quarter |   start_week |   start_weekday |   end_year |   end_quarter |   end_week |   end_weekday |   total_amount |   coupon_covering_ratio% |   customer_covering_ratio% |   amount/day |   redemption_ratio% |\n|---:|--------------:|:----------------|----------------:|-------------:|----------------:|-------------:|----------------:|-----------:|--------------:|-----------:|--------------:|---------------:|-------------------------:|---------------------------:|-------------:|--------------------:|\n|  0 |             1 | Y               |              37 |         2012 |               4 |           50 |               2 |       2013 |             1 |          3 |             4 |            143 |                 1.27021  |                   0.910364 |      3.86486 |            0.699301 |\n|  1 |             2 | Y               |              32 |         2012 |               4 |           51 |           

In [ ]:
df.to_csv('/content/drive/MyDrive/Project Data Mining/regression_campaign.csv')